# 1. 학습데이터 수집

## 1-1. 이름을 입력받아서 분류 수집 : OpenCV2-HaarCascade
이름을 입력 받아서 해당 이름으로 이미 디렉토리가 있으면 해당 디렉토리 사용/없는 경우에는 새로운 디렉토리를 생성해서 해당 디렉토리 내에 사진 수집

In [ ]:
#   opencv python 코딩
#   얼굴을 인식한 상태에만 자동으로 촬영
#

import cv2
from PIL import ImageFont, ImageDraw, Image
import numpy as np
import os
import datetime

# opencv python 코딩 기본 틀
# 카메라 영상을 받아올 객체 선언 및 설정(영상 소스, 해상도 설정)
capture = cv2.VideoCapture(0)
capture.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)


is_record = False   # 녹화상태 : 녹화중 x
on_record = False
cnt_record = 0      # 영상 녹화 시간 관련 변수
max_cnt_record = 1  # 최소 촬영시간

fourcc = cv2.VideoWriter_fourcc(*'XVID')    # 영상을 기록할 코덱 설정
#font = ImageFont.truetype('fonts/ARIAL.TTF', 20) # 글꼴파일을 불러옴

# haar cascade 검출기 객체 선언
# face_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')
face_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_alt.xml')
#eye_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_eye_tree_eyeglasses.xml')

# 이름 입력 받기
name = input("Enter Name :") # 영어로

# 디렉토리 생성 - 이미 존재할 경우 해당 디렉토리 사용
output_path = os.getcwd() + "/capture/" + name + "/"
print(output_path)
os.makedirs(output_path, exist_ok = True)


# 무한루프
while True:
    # 현재시각을 불러와 문자열로저장
    now = datetime.datetime.now()
    nowDatetime = now.strftime('%Y-%m-%d %H:%M:%S')
    nowDatetime_path = now.strftime('%Y-%m-%d %H_%M_%S') # 파일이름으로는 :를 못쓰기 때문에 따로 만들어줌

    ret, frame = capture.read()     # 카메라로부터 현재 영상을 받아 frame에 저장, 잘 받았다면 ret가 참
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # 영상을 흑백으로 바꿔줌
    
    # 글자가 잘보이도록 배경을 넣어줌
    # img는 사각형을 넣을 이미지, pt1, pt2는 사각형의 시작점과 끝점, color는 색상(파랑,초록,빨강), tickness는 선굵기(-1은 내부를 채우는 것)
    cv2.rectangle(img=frame, pt1=(10, 15), pt2=(340, 35), color=(0,0,0), thickness=-1)     
    frame = Image.fromarray(frame)    
    draw = ImageDraw.Draw(frame)    
    # xy는 텍스트 시작위치, text는 출력할 문자열, font는 글꼴, fill은 글자색(파랑,초록,빨강)   
#     draw.text(xy=(10, 15),  text="camera "+nowDatetime, font=font, fill=(255, 255, 255))
    draw.text(xy=(10, 15),  text="camera "+nowDatetime, fill=(255, 255, 255))
    frame = np.array(frame)
    
    # scaleFactor를 1에 가깝게 해주면 정확도가 상승하나 시간이 오래걸림
    # minNeighbors를 높여주면 검출률이 상승하나 오탐지율도 상승
    faces = face_cascade.detectMultiScale(gray, scaleFactor= 1.5, minNeighbors=3, minSize=(20,20))
    # print(faces)
    
    # 찾은 얼굴이 있으면
    if len(faces) :
        is_record = True    # 녹화 중
        if on_record == False: # 
            #video = cv2.VideoWriter("capture/capture" + nowDatetime_path + ".avi", fourcc, 1, (frame.shape[1], frame.shape[0]))
#             cv2.imwrite(os.getcwd() + "/capture/capture" + nowDatetime_path + ".jpg", frame) # 프레임 캡쳐
#             cv2.imwrite("C:/Users/CPB06GameN/Python/face_recognition/capture/재석/capture" + nowDatetime_path + ".jpg", frame) # 프레임 캡쳐
#             cv2.imwrite(os.getcwd() + "/capture/" + name + "/capture" + nowDatetime_path + ".jpg", frame) # 프레임 캡쳐
            cv2.imwrite(output_path + "capture" + nowDatetime_path + ".jpg", frame) # 프레임 캡쳐
    
        cnt_record = max_cnt_record
        
    if is_record == True:   # 녹화중이면
        print('녹화 중')
        #video.write(frame)    # 현재 프레임 저장
        cnt_record = cnt_record - 1     # 녹화시간 1 감소
        on_record = True    # 녹화중 여부를 참으로
    if cnt_record == 0:     # 녹화시간이 다 되면
        is_record = False   # 녹화관련 변수들을 거짓으로
        on_record = False
    
    
    # 얼굴 영역을 영상에 사각형으로 표시
    if len(faces) :
        for  x, y, w, h in faces :
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255,255,255), 2, cv2.LINE_4)
    cv2.imshow("original", frame)   # frame(카메라 영상)을 original 이라는 창에 띄워줌 
    if cv2.waitKey(1000) == ord('q'):  # 키보드의 q 를 누르면 무한루프가 멈춤
            break

capture.release()                   # 캡처 객체를 없애줌
cv2.destroyAllWindows()             # 모든 영상 창을 닫아줌

# 2. 학습데이터 전처리

## 2-1. 얼굴 정보 수집 : face_landmark

In [ ]:
import numpy as np
import dlib
import cv2
import time

RIGHT_EYE = list(range(36, 42))
LEFT_EYE = list(range(42, 48))
MOUTH = list(range(48, 68))
NOSE = list(range(27, 36))
EYEBROWS = list(range(17, 27))
JAWLINE = list(range(1, 17))
ALL = list(range(0, 68))
EYES = list(range(36, 48))

predictor_file = 'model/shape_predictor_68_face_landmarks.dat'
image_file = 'dataset/UZ/11.jpg'

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_file)

image = cv2.imread(image_file)
image_resize = cv2.resize(image, dsize=(700,700), interpolation = cv2.INTER_AREA) #사이즈 수정
gray = cv2.cvtColor(image_resize, cv2.COLOR_BGR2GRAY)

start_time = time.time()
rects = detector(gray, 2) # 5>시간오래 걸리고 과탐지
end_time = time.time()
detect_time = end_time - start_time
print("Number of faces detected: {}".format(len(rects)))
print("=== Detection took {:.3f} seconds".format(detect_time))

for (i, rect) in enumerate(rects):
    start_time = time.time()

    points = np.matrix([[p.x, p.y] for p in predictor(gray, rect).parts()])
    show_parts = points[ALL]
    for (i, point) in enumerate(show_parts):
        x = point[0,0]
        y = point[0,1]
        #cv2.circle(image, (x, y), 1, (0, 255, 255), -1)
        cv2.circle(image_resize, (x, y), 1, (0, 255, 255), -1)
        #cv2.putText(image, "{}".format(i + 1), (x, y - 2),
		#cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 255, 0), 1)
        cv2.putText(image_resize, "{}".format(i + 1), (x, y - 2),
		cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 255, 0), 1)
    end_time = time.time()
    process_time = end_time - start_time
    print("=== A frame took {:.3f} seconds".format(process_time))

cv2.imshow("Face Landmark", image_resize)
cv2.waitKey(0)   


## 2-2. 얼굴 사진 전처리 : face_alignment, face_landmark
face_landmark를 통해 얻은 얼굴 정보에서 눈(eyes)에 해당하는 값들을 numpy_arctan함수를 이용하여 기울기를 계산하고 face_alignment를 사용하여 수평에 맞게 보정 및 기본 cv2함수로 얼굴만 정해놓은 사이즈로 잘라냄

### 2-2-1. ROI(Region of Interest) 설정 : face_landmark, face_alignment
보정된 사진을 이용하여 정확도를 높이고 최적화를 위해서 관심영역을 얼굴로 지정하고 해당 영역만 수집

In [1]:
import numpy as np
import dlib
import cv2
#from google.colab.patches import cv2_imshow #coloab
import datetime



RIGHT_EYE = list(range(36, 42))
LEFT_EYE = list(range(42, 48))
EYES = list(range(36, 48))

predictor_file = 'model/shape_predictor_68_face_landmarks.dat'
image_file = 'dataset/MZ/1.jpg'
MARGIN_RATIO = 1.5
OUTPUT_SIZE = (300, 300)


detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_file)

image = cv2.imread(image_file)
image_origin = image.copy()

(image_height, image_width) = image.shape[:2]
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

rects = detector(gray, 1)

def getFaceDimension(rect):
    return (rect.left(), rect.top(), rect.right() - rect.left(), rect.bottom() - rect.top())

def getCropDimension(rect, center):
    width = (rect.right() - rect.left())
    half_width = width // 2
    (centerX, centerY) = center
    startX = centerX - half_width
    endX = centerX + half_width
    startY = rect.top()
    endY = rect.bottom() 
    return (startX, endX, startY, endY)    

for (i, rect) in enumerate(rects):
    (x, y, w, h) = getFaceDimension(rect)
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

    points = np.matrix([[p.x, p.y] for p in predictor(gray, rect).parts()])
    show_parts = points[EYES]

    right_eye_center = np.mean(points[RIGHT_EYE], axis = 0).astype("int")
    left_eye_center = np.mean(points[LEFT_EYE], axis = 0).astype("int")
    print(right_eye_center, left_eye_center)
 
    cv2.circle(image, (right_eye_center[0,0], right_eye_center[0,1]), 5, (0, 0, 255), -1)
    cv2.circle(image, (left_eye_center[0,0], left_eye_center[0,1]), 5, (0, 0, 255), -1)
    
    cv2.circle(image, (left_eye_center[0,0], right_eye_center[0,1]), 5, (0, 255, 0), -1)
    
    cv2.line(image, (right_eye_center[0,0], right_eye_center[0,1]),
             (left_eye_center[0,0], left_eye_center[0,1]), (0, 255, 0), 2)
    cv2.line(image, (right_eye_center[0,0], right_eye_center[0,1]),
         (left_eye_center[0,0], right_eye_center[0,1]), (0, 255, 0), 1)
    cv2.line(image, (left_eye_center[0,0], right_eye_center[0,1]),
         (left_eye_center[0,0], left_eye_center[0,1]), (0, 255, 0), 1)

    eye_delta_x = right_eye_center[0,0] - left_eye_center[0,0]
    eye_delta_y = right_eye_center[0,1] - left_eye_center[0,1]
    degree = np.degrees(np.arctan2(eye_delta_y,eye_delta_x)) - 180

    eye_distance = np.sqrt((eye_delta_x ** 2) + (eye_delta_y ** 2))
    aligned_eye_distance = left_eye_center[0,0] - right_eye_center[0,0]
    scale = aligned_eye_distance / eye_distance

    eyes_center = ((left_eye_center[0,0] + right_eye_center[0,0]) // 2,
            (left_eye_center[0,1] + right_eye_center[0,1]) // 2)
    cv2.circle(image, eyes_center, 5, (255, 0, 0), -1)
            
    metrix = cv2.getRotationMatrix2D(eyes_center, degree, scale)
    cv2.putText(image, "{:.5f}".format(degree), (right_eye_center[0,0], right_eye_center[0,1] + 20),
     	 cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

    warped = cv2.warpAffine(image_origin, metrix, (image_width, image_height),
        flags=cv2.INTER_CUBIC)
    
    cv2.imshow("warpAffine", warped) # 각도 회전
    #cv2_imshow(warped) #colab

    (startX, endX, startY, endY) = getCropDimension(rect, eyes_center)
    croped = warped[startY:endY, startX:endX]
    output = cv2.resize(croped, OUTPUT_SIZE)
    cv2.imshow("output", output) # 최종
    #cv2_imshow(output) #colab
    
    # 현재시각을 불러와 문자열로저장
    now = datetime.datetime.now()
    nowDatetime = now.strftime('%Y-%m-%d %H:%M:%S')
    nowDatetime_path = now.strftime('%Y-%m-%d %H_%M_%S') # 파일이름으로는 :를 못쓰기 때문에 따로 만들어줌
    #cv2.imwrite("dataset/aligned/MZ/" + nowDatetime_path + ".jpg", output)


    for (i, point) in enumerate(show_parts):
        x = point[0,0]
        y = point[0,1]
        cv2.circle(image, (x, y), 1, (0, 255, 255), -1)

cv2.imshow("Face Alignment", image)
#cv2_imshow(image) #colab



cv2.waitKey(0)   
cv2.destroyAllWindows()
 

[[460 489]] [[705 385]]


### 2-2-2. 학습데이터 전처리 자동화
이름이나 학번/사번을 입력받아 새로운 디렉토리를 생성하고 학습데이터를 폴더별로 조회하여 수집된 사진을 전처리 한 후 생성된 디렉토리에 자동으로 분류

In [ ]:
import numpy as np
import dlib
import cv2
import os

RIGHT_EYE = list(range(36, 42))
LEFT_EYE = list(range(42, 48))
EYES = list(range(36, 48))

dataset_paths = ['/dataset/JB/', '/dataset/MZ/', '/dataset/UZ/', '/dataset/kim min jae/','/dataset/son/','/dataset/tedy/']
output_paths = ['/dataset/aligned/JB/', '/dataset/aligned/MZ/', '/dataset/aligned/UZ/', '/dataset/aligned/kim min jae/', '/dataset/aligned/son/', '/dataset/aligned/tedy/']
image_type = '.jpg'

predictor_file = './model/shape_predictor_68_face_landmarks.dat'
MARGIN_RATIO = 1.5
OUTPUT_SIZE = (300, 300)

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_file)


def getFaceDimension(rect):
    return (rect.left(), rect.top(), rect.right() - rect.left(), rect.bottom() - rect.top())


def getCropDimension(rect, center):
    width = (rect.right() - rect.left())
    half_width = width // 2
    (centerX, centerY) = center
    startX = centerX - half_width
    endX = centerX + half_width
    startY = rect.top()
    endY = rect.bottom() 
    return (startX, endX, startY, endY)    

for (i, dataset_path) in enumerate(dataset_paths):
    output_path = output_paths[i]
    
    ## 파일(이미지 개수)
    dataset_path = dataset_paths[i]
#     print(dataset_path)
#     wd = os.getcwd() + "/" + dataset_path
    dataset_path = os.getcwd() + dataset_path
    output_path = os.getcwd() + output_path
#     print(wd)
    
    jpg_list = [file for file in os.listdir(dataset_path) if file.endswith(image_type)]
    print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
    print(jpg_list)
    number_images = len(jpg_list) # .jpg만
    ##
    
    for idx in range(number_images):
        input_file = dataset_path + str(idx+1) + image_type
#         input_file = dataset_path + str(idx+1) + image_type
        print("-------------------------------------------------------------------------------")
        print("input path: " + input_file)
        # get RGB image from BGR, OpenCV format
        image = cv2.imread(input_file)
        if image is None:
            print("load failed!!\n")
            continue
        image_origin = image.copy()

        (image_height, image_width) = image.shape[:2]
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        rects = detector(gray, 1)

        for (i, rect) in enumerate(rects):
            (x, y, w, h) = getFaceDimension(rect)
            if 'x' in locals():
                print("face detected!!")
            else :
                print("face not detected!!")
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

            # face_landmark로 얼굴 탐지 및 랜드마크 좌표 획득
            points = np.matrix([[p.x, p.y] for p in predictor(gray, rect).parts()])
#             print(type(str(points.ndim)))
#             print("face_landmark: " + str(len(points)))            
            show_parts = points[EYES]

            right_eye_center = np.mean(points[RIGHT_EYE], axis = 0).astype("int")
            left_eye_center = np.mean(points[LEFT_EYE], axis = 0).astype("int")

            eye_delta_x = right_eye_center[0,0] - left_eye_center[0,0]
            eye_delta_y = right_eye_center[0,1] - left_eye_center[0,1]
            degree = np.degrees(np.arctan2(eye_delta_y,eye_delta_x)) - 180

            eye_distance = np.sqrt((eye_delta_x ** 2) + (eye_delta_y ** 2))
            aligned_eye_distance = left_eye_center[0,0] - right_eye_center[0,0]
            scale = aligned_eye_distance / eye_distance

            eyes_center = ((left_eye_center[0,0] + right_eye_center[0,0]) // 2,
                    (left_eye_center[0,1] + right_eye_center[0,1]) // 2)
                    
            metrix = cv2.getRotationMatrix2D(eyes_center, degree, scale)

            warped = cv2.warpAffine(image_origin, metrix, (image_width, image_height),
                flags=cv2.INTER_CUBIC)

            (startX, endX, startY, endY) = getCropDimension(rect, eyes_center)

            croped = warped[startY:endY, startX:endX]
            output = cv2.resize(croped, OUTPUT_SIZE)
            #output = warped[startY:endY, startX:endX]
            
            output_file = output_path + str(idx+1) + image_type
            print("output_path: " + output_file)
            print("saved!!\n")
            cv2.imshow(output_file, output)
            cv2.imwrite(output_file, output)
        
cv2.waitKey(0)   
cv2.destroyAllWindows()


# 3. 얼굴 인식(감지 및 식별)
머신의 성능 문제로 Google_Colab에서 진행

## 3-1. 데이터 학습 : pickle
수집한 학습데이터에서 얼굴을 탐지한 후 해당 얼굴정보를 벡터이미지로 인코딩하여 pickle포맷으로 저장 

In [ ]:
import cv2
import face_recognition
import pickle
import os

# dataset_paths = ['/drive/My Drive/face_recognition/dataset/aligned/JB/', '/drive/My Drive/face_recognition/dataset/aligned/MZ/', '/drive/My Drive/face_recognition/dataset/aligned/UZ/', '/drive/My Drive/face_recognition/dataset/aligned/kim min jae/', '/drive/My Drive/face_recognition/dataset/aligned/son/', '/drive/My Drive/face_recognition/dataset/aligned/tedy/']
dataset_paths = ['/drive/My Drive/face_recognition/dataset/aligned/JB/', '/drive/My Drive/face_recognition/dataset/aligned/MZ/', '/drive/My Drive/face_recognition/dataset/aligned/UZ/']
# names = ['Junseok', 'Minzi', 'Yujin', 'Minjae', 'Son', 'Tedy']
names = ['Junseok', 'Minzi', 'Yujin']
image_type = '.jpg'
encoding_file = 'encodings.pickle'
# Either cnn  or hog. The CNN method is more accurate but slower. HOG is faster but less accurate.
model_method = 'cnn'

# initialize the list of known encodings and known names
knownEncodings = []
knownNames = []

# loop over the image paths
for (i, dataset_path) in enumerate(dataset_paths):
    # extract the person name from names
    name = names[i]
    dataset_path = dataset_paths[i]
    dataset_path = os.getcwd() + dataset_path
    
    jpg_list = [file for file in os.listdir(dataset_path) if file.endswith(image_type)]
    number_images = len(jpg_list)
    print("=====================" + name + "=====================")
    print(str(number_images) + "images found")

    for idx in range(number_images):
        file_name = dataset_path + str(idx+1) + image_type
        print(file_name)

        # load the input image and convert it from BGR (OpenCV ordering)
        # to dlib ordering (RGB)
        image = cv2.imread(file_name)
        if image is None:
            print("load failed!!\n")
            continue
        rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # detect the (x, y)-coordinates of the bounding boxes
        # corresponding to each face in the input image
        boxes = face_recognition.face_locations(rgb,
            model=model_method)

        # compute the facial embedding for the face
        encodings = face_recognition.face_encodings(rgb, boxes)

        # loop over the encodings
        for encoding in encodings:
            # add each encoding + name to our set of known names and
            # encodings
            # print(file_name, name, encoding)
            print("file encoded!!")
            knownEncodings.append(encoding)
            knownNames.append(name)
        
# Save the facial encodings + names to disk
data = {"encodings": knownEncodings, "names": knownNames}
f = open(encoding_file, "wb")
f.write(pickle.dumps(data))
f.close()


## 3-2. 얼굴인식 : face_recognition(cnn or hog)
학습시킨 데이터를 통해 얼굴인식을 수행하고자 하는 대상 이미지를 설정한 후, face_recognition을 통해 인코딩된 데이터와 대상 이미지 속에서 탐지된 얼굴정보를 대조하여 얼굴을 식별

In [ ]:
import cv2
import face_recognition
import pickle
import time
from google.colab.patches import cv2_imshow #coloab

image_file = '/content/drive/My Drive/face_recognition/image/10.jpg'
encoding_file = 'encodings.pickle'
unknown_name = 'Unknown'
# Either cnn  or hog. The CNN method is more accurate but slower. HOG is faster but less accurate.
model_method = 'hog'

def detectAndDisplay(image):
    start_time = time.time()
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # detect the (x, y)-coordinates of the bounding boxes corresponding
    # to each face in the input image, then compute the facial embeddings
    # for each face
    boxes = face_recognition.face_locations(rgb,
        model=model_method)
    encodings = face_recognition.face_encodings(rgb, boxes)

    # initialize the list of names for each face detected
    names = []

    # loop over the facial embeddings
    for encoding in encodings:
        # attempt to match each face in the input image to our known
        # encodings
        matches = face_recognition.compare_faces(data["encodings"],
            encoding)
        name = unknown_name

        # check to see if we have found a match
        if True in matches:
            # find the indexes of all matched faces then initialize a
            # dictionary to count the total number of times each face
            # was matched
            matchedIdxs = [i for (i, b) in enumerate(matches) if b]
            counts = {}

            # loop over the matched indexes and maintain a count for
            # each recognized face face
            for i in matchedIdxs:
                name = data["names"][i]
                counts[name] = counts.get(name, 0) + 1

            # determine the recognized face with the largest number of
            # votes (note: in the event of an unlikely tie Python will
            # select first entry in the dictionary)
            name = max(counts, key=counts.get)
        
        # update the list of names
        names.append(name)

    # loop over the recognized faces
    for ((top, right, bottom, left), name) in zip(boxes, names):
        # draw the predicted face name on the image
        y = top - 15 if top - 15 > 15 else top + 15
        color = (0, 255, 0)
        line = 2
        if(name == unknown_name):
            color = (0, 0, 255)
            line = 1
            name = ''
            
        cv2.rectangle(image, (left, top), (right, bottom), color, line)
        y = top - 15 if top - 15 > 15 else top + 15
        cv2.putText(image, name, (left, y), cv2.FONT_HERSHEY_SIMPLEX,
            0.75, color, line)

    end_time = time.time()
    process_time = end_time - start_time
    print("=== A frame took {:.3f} seconds".format(process_time))
    # show the output image
#    cv2.imshow("Recognition", image)
    cv2_imshow(image) #colab    
    
# load the known faces and embeddings
data = pickle.loads(open(encoding_file, "rb").read())

# load the input image
image = cv2.imread(image_file)
detectAndDisplay(image)

cv2.waitKey(0)
cv2.destroyAllWindows()


# db연동-cam (예시)

## 출근

In [ ]:
#준석

import cv2
from PIL import ImageFont, ImageDraw, Image
import numpy as np
import os
import datetime
import time
import pymysql

capture = cv2.VideoCapture(0)
capture.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

conn = pymysql.connect(host='192.168.0.232', port = 9966, user='root', password='root123', db='hnu_iwaz', charset='utf8', )
curs = conn.cursor(pymysql.cursors.DictCursor)

is_record = False   # 녹화상태 : 녹화중 x
on_record = False
cnt_record = 0      # 영상 녹화 시간 관련 변수
max_cnt_record = 1  # 최소 촬영시간

fourcc = cv2.VideoWriter_fourcc(*'XVID')    # 영상을 기록할 코덱 설정

# haar cascade 검출기 객체 선언
face_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_alt.xml')

# 이름 입력 받기
name = "Junseok Ban"

isRecognition = False
# 무한루프
while True:
    now = datetime.datetime.now()
    now = now - datetime.timedelta(hours=7)
    
    nowDatetime = now.strftime('%Y-%m-%d %H:%M:%S')
    nowDatetime_path = now.strftime('%Y-%m-%d %H_%M_%S') # 파일이름으로는 :를 못쓰기 때문에 따로 만들어줌

    ret, frame = capture.read()     # 카메라로부터 현재 영상을 받아 frame에 저장, 잘 받았다면 ret가 참
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # 영상을 흑백으로 바꿔줌
    
    cv2.rectangle(img=frame, pt1=(10, 15), pt2=(340, 35), color=(0,0,0), thickness=-1)     
    frame = Image.fromarray(frame)    
    draw = ImageDraw.Draw(frame)    
    draw.text(xy=(10, 15),  text="camera "+nowDatetime, fill=(255, 255, 255))
    frame = np.array(frame)
    
    faces = face_cascade.detectMultiScale(gray, scaleFactor= 1.5, minNeighbors=3, minSize=(20,20))
    
    if len(faces) :
        for  x, y, w, h in faces :
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0,255,0), 2, cv2.LINE_4)
            cv2.putText(frame, name, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)
            global isRecongnition
            if isRecognition == False:
                sql = "insert into face(name, time, type) values(\'" + name + "\', \'" + nowDatetime + "\', \'출근\')"
#                 sql = """
#                 insert into face(name, time, type) values('name', 'time.strftime('%Y-%m-%d %H:%M:%S')', '출근')
#                 """
                global conn
                global curs
                curs.execute(sql)
                conn.commit()
                isRecognition = True
    else:
        isRecognition = False
    cv2.imshow("original", frame)   # frame(카메라 영상)을 original 이라는 창에 띄워줌 
    if cv2.waitKey(1000) == ord('q'):  # 키보드의 q 를 누르면 무한루프가 멈춤
            break

capture.release()                   # 캡처 객체를 없애줌
cv2.destroyAllWindows()             # 모든 영상 창을 닫아줌

## 퇴근

In [6]:
#유진

import cv2
from PIL import ImageFont, ImageDraw, Image
import numpy as np
import os
import datetime
import time
import pymysql

capture = cv2.VideoCapture(0)
capture.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

conn = pymysql.connect(host='192.168.0.232', port = 9966, user='root', password='root123', db='hnu_iwaz', charset='utf8', )
curs = conn.cursor(pymysql.cursors.DictCursor)

is_record = False   # 녹화상태 : 녹화중 x
on_record = False
cnt_record = 0      # 영상 녹화 시간 관련 변수
max_cnt_record = 1  # 최소 촬영시간

fourcc = cv2.VideoWriter_fourcc(*'XVID')    # 영상을 기록할 코덱 설정

# haar cascade 검출기 객체 선언
face_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_alt.xml')

# 이름 입력 받기
name = "Yujin Kim"

isRecognition = False
# 무한루프
while True:
    now = datetime.datetime.now()
    now = now + datetime.timedelta(hours=2)
    
    nowDatetime = now.strftime('%Y-%m-%d %H:%M:%S')
    nowDatetime_path = now.strftime('%Y-%m-%d %H_%M_%S') # 파일이름으로는 :를 못쓰기 때문에 따로 만들어줌

    ret, frame = capture.read()     # 카메라로부터 현재 영상을 받아 frame에 저장, 잘 받았다면 ret가 참
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # 영상을 흑백으로 바꿔줌
    
    cv2.rectangle(img=frame, pt1=(10, 15), pt2=(340, 35), color=(0,0,0), thickness=-1)     
    frame = Image.fromarray(frame)    
    draw = ImageDraw.Draw(frame)    
    draw.text(xy=(10, 15),  text="camera "+nowDatetime, fill=(255, 255, 255))
    frame = np.array(frame)
    
    faces = face_cascade.detectMultiScale(gray, scaleFactor= 1.5, minNeighbors=3, minSize=(20,20))
    
    if len(faces) :
        for  x, y, w, h in faces :
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0,255,0), 2, cv2.LINE_4)
            cv2.putText(frame, name, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)
            global isRecongnition
            if isRecognition == False:
                sql = "insert into face(name, time, type) values(\'" + name + "\', \'" + nowDatetime + "\', \'퇴근\')"
#                 sql = """
#                 insert into face(name, time, type) values('name', 'time.strftime('%Y-%m-%d %H:%M:%S')', '출근')
#                 """
                global conn
                global curs
                curs.execute(sql)
                conn.commit()
                isRecognition = True
    else:
        isRecognition = False
    cv2.imshow("original", frame)   # frame(카메라 영상)을 original 이라는 창에 띄워줌 
    if cv2.waitKey(1000) == ord('q'):  # 키보드의 q 를 누르면 무한루프가 멈춤
            break

capture.release()                   # 캡처 객체를 없애줌
cv2.destroyAllWindows()             # 모든 영상 창을 닫아줌